# Getting Started with GANDALF

This tutorial introduces the basics of running magnetohydrodynamic turbulence simulations with GANDALF.

**What you'll learn:**
- Setting up a spectral grid
- Initializing turbulent states
- Running forced turbulence simulations
- Monitoring energy evolution
- Basic visualization

**Runtime:** ~2 seconds on M1 Pro / modern laptop

## Setup and Imports

First, let's import the necessary modules. GANDALF is built on JAX for high-performance computing.

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from krmhd import (
    SpectralGrid3D,
    initialize_alfven_wave,
    gandalf_step,
    force_alfven_modes,
    compute_energy_injection_rate,
    energy as compute_energy,
)

# Set matplotlib style
plt.style.use('default')
%matplotlib inline

## 1. Create a Spectral Grid

GANDALF uses **3D Fourier spectral methods** for spatial discretization. We create a grid with:
- `Nx`, `Ny`: Perpendicular resolution (tangent to magnetic field)
- `Nz`: Parallel resolution (along magnetic field)

The grid uses `rfft` format (real FFT) for memory efficiency.

In [ ]:
# Create a 32x32x16 grid (low resolution for fast demo)
grid = SpectralGrid3D.create(Nx=32, Ny=32, Nz=16)

print(f"Grid shape: {grid.Nx} × {grid.Ny} × {grid.Nz}")
print(f"Box size: Lx={grid.Lx:.2f}, Ly={grid.Ly:.2f}, Lz={grid.Lz:.2f}")
print(f"Fourier space shape: {grid.kx.shape}")

## 2. Initialize the State

We'll start with an **Alfvén wave** - a fundamental MHD wave mode that propagates along the magnetic field.

The state contains:
- `z_plus`, `z_minus`: Elsasser variables (combinations of velocity and magnetic field)
- Optionally: Hermite moments for kinetic physics (not used in this basic example)

In [ ]:
# Initialize Alfvén wave with amplitude 0.01
# M=10 Hermite moments (set to 0 for fluid limit)
# kz_mode=1 means one wavelength along z
state = initialize_alfven_wave(grid, M=10, kz_mode=1, amplitude=0.01)

# Check initial energy
E0 = compute_energy(state)
print(f"Initial total energy: {E0['total']:.4e}")
print(f"  Magnetic: {E0['magnetic']:.4e}")
print(f"  Kinetic:  {E0['kinetic']:.4e}")

## 3. Set Physics Parameters

Key parameters for turbulence simulations:

- **`dt`**: Timestep (must satisfy CFL condition: dt < dx/v_A)
- **`eta`**: Resistivity (magnetic diffusion coefficient)
- **`v_A`**: Alfvén velocity (normalized to 1.0)
- **`amplitude`**: Forcing amplitude (energy injection rate)
- **`n_min`, `n_max`**: Mode numbers for forcing (large scales)

In [ ]:
# Timestep and physics parameters
dt = 0.01          # Timestep (0.01 τ_A)
eta = 0.01         # Resistivity
v_A = 1.0          # Alfvén velocity (normalized)

# Forcing parameters
forcing_amplitude = 0.1  # Energy injection amplitude
n_min = 1                # Minimum mode number (largest scale)
n_max = 2                # Maximum mode number

# Initialize random key for stochastic forcing
key = jax.random.PRNGKey(42)

print("Parameters set:")
print(f"  dt = {dt} τ_A")
print(f"  η = {eta}")
print(f"  Forcing modes: n = {n_min}-{n_max}")

## 4. Run the Simulation

The simulation loop consists of:
1. **Forcing**: Inject energy at large scales (low mode numbers)
2. **Evolution**: Nonlinear cascade + dissipation via `gandalf_step()`
3. **Diagnostics**: Track energy and injection rate

Let's run for 50 steps and track the evolution.

In [ ]:
# Storage for diagnostics
times = []
energies = []
injection_rates = []

# Initial energy
times.append(0.0)
energies.append(E0['total'])
injection_rates.append(0.0)

# Run simulation
n_steps = 50
print(f"Running {n_steps} steps...\n")

for i in range(n_steps):
    # 1. Apply forcing
    state_before = state
    state, key = force_alfven_modes(
        state,
        amplitude=forcing_amplitude,
        n_min=n_min,
        n_max=n_max,
        dt=dt,
        key=key
    )
    
    # 2. Measure energy injection
    eps_inj = compute_energy_injection_rate(state_before, state, dt)
    
    # 3. Evolve dynamics
    state = gandalf_step(state, dt=dt, eta=eta, v_A=v_A)
    
    # 4. Diagnostics
    E = compute_energy(state)['total']
    t = (i + 1) * dt
    
    times.append(t)
    energies.append(E)
    injection_rates.append(eps_inj)
    
    if (i + 1) % 10 == 0:
        print(f"Step {i+1:3d}: t = {t:.2f} τ_A, E = {E:.4e}, ε_inj = {eps_inj:.3e}")

print(f"\nSimulation complete!")
print(f"Final energy: {energies[-1]:.4e} (initial: {E0['total']:.4e})")

## 5. Visualize Energy Evolution

Let's plot how the total energy evolves over time. In **forced turbulence**, we expect:
- Energy injection from forcing at large scales
- Nonlinear cascade transferring energy to small scales
- Dissipation at small scales via resistivity
- Eventual **statistical steady state** where injection ≈ dissipation

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Plot 1: Energy evolution
ax1.plot(times, energies, 'b-', linewidth=2)
ax1.axhline(E0['total'], color='gray', linestyle='--', label='Initial energy')
ax1.set_xlabel('Time (τ_A)', fontsize=12)
ax1.set_ylabel('Total Energy', fontsize=12)
ax1.set_title('Energy Evolution in Forced Turbulence', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot 2: Energy injection rate
ax2.plot(times[1:], injection_rates[1:], 'r-', linewidth=1.5, alpha=0.7)
ax2.axhline(jnp.mean(jnp.array(injection_rates[1:])), color='darkred', 
            linestyle='--', label=f'Mean: {jnp.mean(jnp.array(injection_rates[1:])):.3e}')
ax2.set_xlabel('Time (τ_A)', fontsize=12)
ax2.set_ylabel('Energy Injection Rate ε', fontsize=12)
ax2.set_title('Stochastic Energy Injection', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print(f"\nAverage injection rate: {jnp.mean(jnp.array(injection_rates[1:])):.3e}")
print(f"Energy change: {(energies[-1] - E0['total'])/E0['total']*100:.1f}%")

## 6. Understanding the Results

What's happening physically?

1. **Forcing** injects energy randomly at large scales (modes n=1-2)
2. **Nonlinear interactions** cascade energy to smaller scales via Poisson brackets
3. **Resistivity** dissipates energy at small scales (high wavenumbers)
4. The system approaches a **statistical steady state** where injection ≈ dissipation

For this short run, the energy is still growing - it needs longer to reach steady state.

### Next Steps

Try modifying the parameters:
- Increase `n_steps` to 200-500 to see steady state
- Change `forcing_amplitude` to adjust energy injection
- Modify `eta` to change dissipation rate
- Increase resolution (e.g., 64×64×32) for better turbulence

### Advanced Topics

See the other tutorials:
- **02_driven_turbulence.ipynb**: Comprehensive diagnostics, energy spectra
- **03_analyzing_decay.ipynb**: Decaying turbulence and spectral analysis

## Optional: Quick Parameter Sensitivity Test

Let's run a quick comparison of different forcing amplitudes:

In [ ]:
# Run multiple simulations with different forcing
amplitudes_to_test = [0.05, 0.1, 0.2]
results = {}

print("Testing different forcing amplitudes...\n")

for amp in amplitudes_to_test:
    # Reset state
    state_test = initialize_alfven_wave(grid, M=10, kz_mode=1, amplitude=0.01)
    key_test = jax.random.PRNGKey(42)
    
    energies_test = [compute_energy(state_test)['total']]
    
    # Run simulation
    for i in range(30):
        state_test, key_test = force_alfven_modes(
            state_test, amplitude=amp, n_min=1, n_max=2, dt=dt, key=key_test
        )
        state_test = gandalf_step(state_test, dt=dt, eta=eta, v_A=v_A)
        energies_test.append(compute_energy(state_test)['total'])
    
    results[amp] = energies_test
    print(f"Amplitude {amp:.2f}: Final E = {energies_test[-1]:.4e}")

# Plot comparison
plt.figure(figsize=(10, 6))
for amp, energies_test in results.items():
    plt.plot(jnp.arange(len(energies_test)) * dt, energies_test, 
             linewidth=2, label=f'Amplitude = {amp}')

plt.xlabel('Time (τ_A)', fontsize=12)
plt.ylabel('Total Energy', fontsize=12)
plt.title('Effect of Forcing Amplitude on Energy Evolution', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("\nHigher forcing amplitude → faster energy growth!")

## Summary

In this tutorial, you learned:

✅ How to create a spectral grid for MHD simulations  
✅ Initialize turbulent states with Alfvén waves  
✅ Run forced turbulence simulations with energy injection  
✅ Track energy evolution and injection rates  
✅ Visualize simulation results  
✅ Test parameter sensitivity  

### Where to go next?

- **Tutorial 02**: Comprehensive driven turbulence with energy spectra
- **Tutorial 03**: Analyzing decaying turbulence and spectral slopes
- **Examples**: Check `examples/benchmarks/` for more complex simulations
- **Documentation**: See `docs/` for physics background and numerical methods